In [ ]:
!brew install tesseract
!brew install tesseract-lang
!brew install poppler

## Описание документа:

https://theswissbay.ch/pdf/Books/Linguistics/Mega%20linguistics%20pack/Uralic/Moksha%3B%20%D0%A3%D1%87%D0%B8%D0%BC%D1%81%D1%8F%20%D0%B3%D0%BE%D0%B2%D0%BE%D1%80%D0%B8%D1%82%D1%8C%20%D0%BF%D0%BE%20%D0%BC%D0%BE%D0%BA%D1%88%D0%B0%D0%BD%D1%81%D0%BA%D0%B8%20%28Poljakov%29.pdf

сначала предполагалось использование OCR для распознавания страниц учебника, но потом был найден цифровой вариант учебника. Словарь остался тем же от OCR, а все параграфы использовались из сайта

https://e-trans.ru/languages/about.php?n=118

в ходе OCR словарей, все примеры, где для одного слова несколько переводов (пусть n), превращались в n строк по 1-1 

## Обработка текстового файла, полученного с помощью копирования материалов параграфов учебника

In [ ]:
import pandas as pd

In [1]:
with open("sources/e_trans.txt", 'r') as file:
    book_data = file.read().split('\n')

In [2]:
all_pairs = []
for example in book_data:
    ru, mdf = example.split(' - ')

    ru = clean_text(ru)
    mdf = clean_text(mdf)

    if not is_text_valid(ru) or is_text_valid(mdf):
        continue

    all_pairs.append({
        'ru': ru,
        'mdf': mdf,
        'source': 'book'
    })

In [3]:
len(all_pairs)

1520

In [5]:
df = pd.DataFrame(all_pairs)

In [6]:
df.sample(10)

,ru,mdf,source
380,память,лятфтама,book
743,Мордва издавна собирает калину и черемуху.,Мокшэрзятне кунара кочксихть чивкт и лаймарьхть.,book
1345,Сладкий.,Ламбама.,book
72,не,аф,book
1279,Возьми обратно!,Сявк меки!,book
1032,Удочкой.,Ульмоса.,book
193,Вы женаты?,Тинь рьвяяфтфтада?,book
571,Оттепель.,Солай.,book
334,За дружбу!,Ялгаксшинкса!,book
402,Надо подумать.,Эряви арьсемс.,book


In [7]:
df['words_count'] = df.apply(lambda x: min([len(x['mdf'].split()), len(x['ru'].split())]), axis=1)

In [8]:
word_df = df[df['words_count'] == 1][['ru', 'mdf', 'source']].reset_index(drop=True)

In [9]:
word_df.sample(10)

,ru,mdf,source
280,воскресенье,недляши,book
217,моему учителю!,тонафтыезенди!,book
226,дедушке!,атязенди,book
790,Хлеб,Кши,book
32,ЗНАКОМСТВО,СОДАВИКС ТИЕМАСЬ,book
688,Здоровый.,Шумбра.,book
739,Холодный.,Кельме.,book
563,лапти,карьхть,book
452,малина,инези,book
177,тесть,атя,book


In [10]:
word_df.shape

(797, 3)

In [11]:
word_df = word_df.drop_duplicates().reset_index(drop=True)

In [12]:
word_df.shape

(728, 3)

In [13]:
word_df.to_csv('results/book_words.tsv', sep='\t')

In [14]:
phrase_df = df[df['words_count'] != 1][['ru', 'mdf', 'source']].reset_index(drop=True)

In [15]:
phrase_df.head()

,ru,mdf,source
0,ВСТРЕЧА И ПРИВЕТСТВИЕ,ВАСЕДЕМАСЬ И ШУМБРАКСТОМАСЬ,book
1,Я встречу друга.,Мон васьфтьса ялгазень,book
2,Мы встретимся?,Минь васеттяма?,book
3,Мы встретились.,Минь васедеме.,book
4,Как дела?,Кода тефне?,book


In [16]:
phrase_df.shape

(723, 3)

In [17]:
phrase_df = phrase_df.drop_duplicates()

In [18]:
phrase_df.shape

(713, 3)

In [19]:
phrase_df.sample(10)

,ru,mdf,source
567,Кого ищешь?,Кинь вешендьсак?,book
290,В Мордовии много лесов.,Мордбвияса лама вирьда.,book
493,Я тебя понимаю.,Мон тонь шарьхкотте.,book
96,"Да, хорошая.","Да, цебярь.",book
344,"Можно попить молоко, сок, компот, кофе.","Симомс ули лофца, сок, компот, кофе.",book
386,В нашем селе живет 2000 человек.,Минь велесонк эряй кафта тёжятть ломань.,book
199,Не забудьте.,Тясть юксне.,book
490,Что он хочет?,Мезе сон ёрай?,book
20,Кто ты? Кто вы?,Кият тон? Китада тинь?,book
330,Веленьконди макссть лия лем,Нашему селу дали другое название,book


In [20]:
phrase_df.to_csv('results/book_phrases.tsv', sep='\t')

## Обработка словарей (OCR)

In [21]:
from pdf2image import convert_from_path
import os
from PIL import Image
import pytesseract

pdf_path = "sources/dicts.pdf"
images = convert_from_path(pdf_path)

In [22]:
import re

In [23]:
texts = []

for image in images:
    text = pytesseract.image_to_string(image, lang='rus')
    text = re.sub(r'\n+', ' ', text)

    texts.append(text)

In [24]:
print('\n\n'.join(texts))

РУССКО-МОКШАНСКИЙ СЛОВАРЬ РУЗОНЬ-МОКШЕНЬ ВАЛКС А азбука азбука, -т алфавит алфавит, -т амбар уто/м; -пт ахать ахксне/мс, -сь Б бабушка (по матери) щава, -т (по отцу) баба, -т базар база/р, -рхт барыня баярава, -т бег ласькома бегать ласьконд/емс, -сь беда зия/н, -тт бедный ашу бежать ласьк/омс, -сь без передается изъятельным падежом: ярмак-фтома «без денег» безвкусный аф танцти бездетный идьфтема безденежный ярмакфтома безошибочно эльбятьксфтома безумие ёньфтомаши безумно ёньфтома безумный ёньфтома белка ур, -рхт белый акша беречь ванфт/омс, -сь бесплатно питнефтома бешеный пяляз бить пикс/омс, -сь блестеть пиндолд/омс, -сь ближний маластонь близко маласа близнецы кафтасть блин пача, -т бог шка/й, -йхть богатый козя болезнь урма, -т болеть сяряд/емс, -сь борода сака/л, -лхт- бородавка циль/ге, -кт борона инзама, -т босиком кяпе бояться пел/емс, -сь брага поза браниться сялонд/омс, -сь брат брат, альняка (если стар- ше себя) брить нара/мс, -сь бросить ёрда/мс, -сь бугор пан/да, -тт буде

In [25]:
import json

with open('sources/parsed_book_dicts.txt', 'r') as file:
    pairs = json.load(file)

In [26]:
for pair in pairs:
    if list(pair.keys())[0] == 'ru':
        source = 'ru-mdf-dict'
    else:
        source = 'mdf-ru-dict'

    pair['source'] = source

In [27]:
for pair in pairs:
    pair['mdf'] = re.sub(';', ',', pair['mdf'])
    pair['mdf'] = pair['mdf'].split(',')[0]
    pair['mdf'] = re.sub('/', '', pair['mdf'])

In [28]:
import pandas as pd

In [29]:
df = pd.DataFrame(pairs)

In [30]:
df.shape

(3058, 3)

In [31]:
df = df.drop_duplicates(subset=['ru', 'mdf'])

In [32]:
df.shape

(2561, 3)

In [33]:
df.sample(10)

,ru,mdf,source
0,азбука,азбука,ru-mdf-dict
1,алфавит,алфавит,ru-mdf-dict
2,амбар,утом,ru-mdf-dict
3,ахать,ахкснемс,ru-mdf-dict
4,бабушка,щава,ru-mdf-dict
...,...,...,...
3048,холодный,якшама,mdf-ru-dict
3049,холод,якшама,mdf-ru-dict
3050,пешком,ялга,mdf-ru-dict
3052,каша,ям,mdf-ru-dict


In [34]:
df.to_csv('results/book_dicts.tsv', sep='\t')